In [2]:
from statsmodels.regression.linear_model import OLS
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
import pandas as pd
pd.set_option("display.max_columns", 500)

# Import Data

In [3]:
conn = sqlite3.connect("eia")
cursor = conn.cursor()


In [4]:
df = pd.DataFrame(cursor.execute("""select * from combined2""").fetchall())
df.columns = [description[0] for description in cursor.description]

In [5]:
df = df.drop(columns=["index"])

In [6]:
df["year_month"] = df.apply(lambda x: str(x["year"]) + "-" + str(x["month"]).zfill(2), axis=1)

# Cleaning new fields

In [7]:
df["total_fuel_consumption_mmbtu"] = df["total_fuel_consumption_mmbtu"].apply(lambda x: str(x).replace(",", ""))
df["total_fuel_consumption_mmbtu"] = df["total_fuel_consumption_mmbtu"].astype(int)

In [8]:
def none_to_zero(x):
    if (x=="None") or (x == "."):
        return 0
    else:
        return x

In [9]:
fields_to_clean = ["total_fuel_consumption_quantity", "electric_fuel_consumption_quantity", "total_fuel_consumption_mmbtu", "elec_fuel_consumption_mmbtu", "net_generation_megawatthours", "quantity_gen", "elec_quantity_gen", "tot_mmbtu_gen", "elec_mmbtu_gen", "netgen_gen"]


In [10]:
for fld in fields_to_clean:
    df[fld] = df[fld].apply(lambda x: str(x).replace(",", ""))

In [11]:
for fld in fields_to_clean:
    df[fld] = df[fld].apply(none_to_zero)


In [12]:
for fld in fields_to_clean:
    df[fld] = df[fld].apply(lambda x: str(x).replace(",", ""))


In [13]:
for fld in fields_to_clean:
    df[fld] = df[fld].astype(int)

In [14]:
# if fuel_cost isnt defined, its no use to us
df = df[df.fuel_cost.notna()].copy()

In [15]:
def try_float(x):
    if x =="" or x == None or x == ".":
        return 0
    else:
        return float(x)
df["mmbtu_per_unit_gen"] = df["mmbtu_per_unit_gen"].apply(try_float)

In [16]:
df["mmbtu_per_unit_gen"] = df["mmbtu_per_unit_gen"].astype(float)

In [17]:
df["operator_id"] = df["operator_id"].apply(lambda x: x.replace("operator_", ""))

In [18]:
df["operator_id"] = df["operator_id"].apply(lambda x: int(round(float(x))))

In [20]:
for col in df.columns:
    if "_id" in col:
        try:
            df[col] = df[col].astype(int)
        except:
            print(col)

coalmine_msha_id
nuclear_unit_id
operator_id_gen


In [23]:
df.coalmine_msha_id = df.coalmine_msha_id.fillna(value=0)
df.nuclear_unit_id = df.nuclear_unit_id.fillna(value=0)
df.operator_id_gen = df.operator_id_gen.fillna(value=0)



In [24]:
for col in df.columns:
    if "_id" in col:
        try:
            df[col] = df[col].astype(int)
        except:
            print(col)

In [27]:
df.to_sql("combined4", conn)

In [25]:
len(df)

440666